### Hasan Roknabady

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from copy import deepcopy
import heapq

### Neuron Model Class

### LIF Model used : 


In [5]:
class LIF:
    def __init__(self, R, tau, u_rest, thre, u_spike, u_reset):
        self.dataPass = []
        self.spikes = []
        self.I = []
        self.R = R
        self.tau = tau
        self.u = u_rest
        self.u_rest = u_rest
        self.thre = thre
        self.u_spike = u_spike
        self.u_reset = u_reset

    def movement(self, cir_size=2):
        tmp = 0
        for i in range(self.inhi_n):
            self.p.append(Neuron(t="inhibitory", idx=tmp))
            tmp += 1
        for i in range(self.exci_n):
            self.p.append(Neuron(t="excitatory", idx=tmp))
            tmp += 1

        first_list = []
        for i in range(len(self.p[0].step)):
            first_list.append([-1 for x in range(self.N)])
        first_list = [[-1 for x in range(self.N)] for y in range(len(self.p[0].step))]

        rand_i = Rand_current()
        first_list_idx = []
        for i in range(len(self.p[0].step)):
            first_list_idx.append(rand_i.acquire(i))

        for i in (range(len(self.p[0].step))):
            current_i = first_list_idx[i]
            for g in self.p:
                result = g.bring_up_to_date(j=i, charge=current_i)
                if result > 0:
                    first_list[i][g.idx] = g.idx
                    for k in range(self.N):
                        self.p[k].update_potential(self.w * result, j=i, pre_t=g.t)



    def action(self, I_func, span, dt, unset=True):
        if not unset:
            pass
        else:
            self.rest()
        size = math.ceil(span / dt)
        U = np.zeros(shape=(size, 2))
        spikes = []
        time = 0
        for idx in range(len(U)):
            U[idx, 1] = self.u
            U[idx, 0] = time
            if self.u <= self.thre:
                pass
            else:
                spikes.append(time)
                self.unset()
            du = dt * (-1 * (self.u - self.u_rest)
                       + 1e-3 * self.R * I_func(time)) / self.tau
            self.u += du
            time += dt
        return {'voltage': U, 'spikes': spikes}

    def repeat(self, I_range, span, dt):
        data = np.zeros(shape=(len(I_range), 2))
        for index in range(len(I_range)):
            self.rest()
            Func = lambda x: I_range[index]
            result = self.action(Func, span=span, dt=dt)
            result = result['spikes']
            data[index, 0] = I_range[index]
            if len(result) == 0:
                data[index, 1] = 0
            elif len(result) == 1:
                data[index, 1] = 1 / span
            else:
                data[index, 1] = (len(result) - 1)\
                                 / (result[-1] - result[0])
        return data

    def one_step(self, input_I, time, dt):
        du = dt * (-1 * (self.u - self.u_rest)
                   + 1e-3 * self.R * input_I) / self.tau

        self.u += du
        time += dt
        self.dataPass.append((self.u, time))
        self.I.append((input_I, time))
        if self.u > self.thre:
            self.spikes.append(time)
            self.unset()

    def unset(self):
        self.u = self.u_reset

    def rest(self):
        self.u = self.u_rest

    def dell(self):
        self.dataPass = []
        self.spikes = []
        self.I = []
        self.u = self.u_rest

### Population:

In [6]:
#c -> connections
class Population:
    def __init__(self, p_type, n_list, time_c, j=20):
        self.p_activity = []
        self.n_list = n_list
        self.c = np.zeros((len(n_list), len(n_list)))
        self.j = j
        self.c_type = self.fully_c
        set_c = self.c_type
        self.p_type = p_type
        set_c()
        self.time_c = time_c
        self.c_history = deepcopy(self.c.ravel())

    def fully_c(self):
        self.c = np.ones_like(self.c) * (self.j / len(self.n_list))
        self.inhib_m()

    def cal_activity_history(self, time, dt, threshold):
        activity_list = np.zeros((len(self.n_list), 1))

        for idx in range(len(self.n_list)):
            activity_list[idx, 0] = \
                self.cal_activity_pass_single(idx, time, dt, threshold)
        return activity_list

    def cal_activity_pass_single(self, idx, time, dt, thre):
        neuron = self.n_list[idx]
        S = 0
        activity = 0
        while not self.time_c(S) <= thre:
            if (time - S) in neuron.spikes:
                activity += self.time_c(S)
            S += dt
        return activity

    def one_step(self, input_current,
                 self_activity, time,
                 dt, time_course_thre):

        inputs = self.c.dot(self_activity)
        for i, neuron in enumerate(self.n_list):
            neuron.one_step(input_current + inputs[i, 0], time, dt)
        activity = self.cal_activity_history(time + dt, dt, time_course_thre)
        return activity

    def reset(self, reset_connection=False):
        self.p_activity = []
        if reset_connection:
            set_c = self.c_type
            set_c()
        for n in self.n_list:
            n.dell()

    def inhib_m(self):
        if self.p_type == 'inhibitory':
            self.c = -1 * self.c

    def raster_p(self):
        ax = []
        ay = []
        for neuron in self.n_list:
            id = '#' + str(int(np.floor(np.random.rand() * 10000000)))
            for l in neuron.spikes:
                ax.append(l)
                ay.append(id)
        return ax, ay

    def running(self, I_lists, time_interval, step,
                dt_minus, dt_plus, a_minus,
                a_plus, time_course_threshold):
        learn = self.STDP
        time = 0
        activity = np.zeros((len(self.n_list), 1))
        while time <= time_interval:
            inputs = self.c.dot(activity)
            for i, neuron in enumerate(self.n_list):
                neuron.one_step(I_lists[i](time) + inputs[i, 0], time, step)
            activity = self.cal_activity_history(time + step, step, time_course_threshold)
            learn(time + step, dt_minus, dt_plus, a_minus, a_plus)
            time += step

    def STDP(self, time, dt_minus, dt_plus, a_minus, a_plus):
        new_c = deepcopy(self.c)
        for i in range(len(self.n_list)):
            for j in range(len(self.n_list)):
                if i == j:
                    continue

                pre = self.n_list[j]
                post = self.n_list[i]
                if len(pre.spikes) == 0 \
                        or len(post.spikes) == 0 \
                        or post.spikes[len(post.spikes) - 1] != time:
                    pass

                else:
                    if post.spikes[len(post.spikes) - 1] <= pre.spikes[
                        len(pre.spikes) - 1]:
                        continue

                    dt = abs(post.spikes[len(post.spikes) - 1] -
                             pre.spikes[len(pre.spikes) - 1])
                    new_c[i, j] += a_plus * math.exp(-(dt / dt_plus))
                if len(pre.spikes) == 0 \
                        or len(post.spikes) == 0 \
                        or pre.spikes[
                    len(pre.spikes) - 1] != time:
                    continue

                if post.spikes[len(post.spikes) - 1] < pre.spikes[
                    len(pre.spikes) - 1]:
                    dt = abs(post.spikes[len(post.spikes) - 1] -
                             pre.spikes[len(pre.spikes) - 1])
                    new_c[i, j] += a_minus * math.exp(-(dt / dt_minus))
                    if new_c[i, j] < 0:
                        new_c[i, j] = 0

        self.c_history = np.vstack((self.c_history, new_c.ravel()))
        self.c = deepcopy(new_c)